In [1]:
import scrapy
import csv
import os



In [2]:
class AlibabaCrawlerSpider(scrapy.Spider):
    name = 'alibaba_crawler'
    allowed_domains = ['alibaba.com']
    start_urls = ['http://alibaba.com/']
    
    
    
    def start_requests(self):
    #Read keywords from keywords file and construct the search URL
    
        with open(os.path.join(os.path.dirname(__file__), "keywords.csv")) as search_keywords:
            for keyword in csv.DictReader(search_keywords):
                search_text=keyword["keyword"]
                print(search_text)
                url="https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&CatId=&SearchText={0}&viewtype=G".format(
                    search_text)
    # The meta is used to send our search text into the parser as metadata
                yield scrapy.Request(url, callback = self.parse, meta = {"search_text": search_text})

In [3]:
search_text

NameError: name 'search_text' is not defined

In [4]:
with open(os.path.join(os.path.dirname(__file__), "keywords.csv")) as search_keywords:
            for keyword in csv.DictReader(search_keywords):
                search_text=keyword["keyword"]
                print(search_text)

NameError: name '__file__' is not defined

In [5]:
with open("keywords.csv") as search_keywords:
            for keyword in csv.DictReader(search_keywords):
                search_text=keyword["keyword"]
                print(search_text)

In [6]:
search_keywords

<_io.TextIOWrapper name='keywords.csv' mode='r' encoding='UTF-8'>